In [253]:
# Importing libraries
import json
import numpy as np

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Dense,ReLU
from tensorflow.keras import Sequential

In [254]:
# Load json data
with open("traindata.json","r") as file:
    Data = json.load(file)    

In [255]:
# Placeholder for storing values
all_patterns = []
tags = []
xy = []
sequences_x = []
sequences_y = []

In [256]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return " ".join(filtered_tokens)

In [257]:
for question in Data['questions']:
    tag = question['tag']
    tags.append(tag)
    
    for pattern in question['keywords']:
        all_patterns.append(pattern)
        xy.append((pattern,tag))

In [258]:
all_patterns_dict = {word:i for i,word in enumerate(all_patterns)}

In [259]:
def bag_of_words(processed_sentence,all_patterns):
    tokenized_sentence = [w for w in processed_sentence.split(" ")]
    bag = np.zeros(len(all_patterns),dtype=np.float32)
    
    for w in tokenized_sentence:
        if w in all_patterns:
            bag[all_patterns_dict[w]] = 1.0
            
    return bag

In [260]:
for pattern in xy:
    sequences_x.append(bag_of_words(pattern[0],all_patterns))
    sequences_y.append(pattern[1])
    
sequences_x = np.array(sequences_x)
sequences_y = np.array(sequences_y)

In [261]:
label_to_index = {tag:i for i,tag in enumerate(set(tags))}
labels = np.array([label_to_index[label[1]] for label in xy], dtype=np.int32)

In [262]:
def create_neural_net(input_size,hidden_size, num_classess):
    model = Sequential()
    model.add(Dense(hidden_size,input_shape=(input_size,),activation='relu'))
    model.add(Dense(hidden_size,activation='relu'))
    model.add(Dense(num_classess,activation='softmax'))
    
    return model

In [263]:
Model = create_neural_net(len(sequences_x[0]),8,3)

In [264]:
Model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 8)                 304       
                                                                 
 dense_19 (Dense)            (None, 8)                 72        
                                                                 
 dense_20 (Dense)            (None, 3)                 27        
                                                                 
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [265]:
Model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

In [266]:
Model.fit(sequences_x, labels, epochs=1000, batch_size=4, callbacks=[early_stopping])

Epoch 1/1000
10/10 [==============================] - 1s 3ms/step - loss: 1.1120 - accuracy: 0.3243
Epoch 2/1000
10/10 [==============================] - 0s 6ms/step - loss: 1.1064 - accuracy: 0.3243
Epoch 3/1000
10/10 [==============================] - 0s 6ms/step - loss: 1.1013 - accuracy: 0.2973
Epoch 4/1000
10/10 [==============================] - 0s 5ms/step - loss: 1.0980 - accuracy: 0.3514
Epoch 5/1000
10/10 [==============================] - 0s 5ms/step - loss: 1.0934 - accuracy: 0.3514
Epoch 6/1000
10/10 [==============================] - 0s 4ms/step - loss: 1.0902 - accuracy: 0.4054
Epoch 7/1000
10/10 [==============================] - 0s 5ms/step - loss: 1.0874 - accuracy: 0.4054
Epoch 8/1000
10/10 [==============================] - 0s 5ms/step - loss: 1.0833 - accuracy: 0.4324
Epoch 9/1000
10/10 [==============================] - 0s 4ms/step - loss: 1.0801 - accuracy: 0.4595
Epoch 10/1000
10/10 [==============================] - 0s 5ms/step - loss: 1.0767 - accuracy: 0.4595

10/10 [==============================] - 0s 3ms/step - loss: 0.6273 - accuracy: 0.8919
Epoch 83/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.6183 - accuracy: 0.8919
Epoch 84/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.6097 - accuracy: 0.8919
Epoch 85/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.5987 - accuracy: 0.8919
Epoch 86/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.5908 - accuracy: 0.8919
Epoch 87/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.5818 - accuracy: 0.8919
Epoch 88/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.5741 - accuracy: 0.8919
Epoch 89/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.5650 - accuracy: 0.8919
Epoch 90/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.5565 - accuracy: 0.8919
Epoch 91/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.5454 - accuracy: 0.8919
Epoc

10/10 [==============================] - 0s 4ms/step - loss: 0.2515 - accuracy: 0.9189
Epoch 163/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.2517 - accuracy: 0.8919
Epoch 164/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.2473 - accuracy: 0.9189
Epoch 165/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.2450 - accuracy: 0.9189
Epoch 166/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.2426 - accuracy: 0.9189
Epoch 167/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.2406 - accuracy: 0.9189
Epoch 168/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.2394 - accuracy: 0.8919
Epoch 169/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.2362 - accuracy: 0.9189
Epoch 170/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.2355 - accuracy: 0.9189
Epoch 171/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.2335 - accuracy: 0.

10/10 [==============================] - 0s 4ms/step - loss: 0.1278 - accuracy: 0.9459
Epoch 243/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.1268 - accuracy: 0.9459
Epoch 244/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.1270 - accuracy: 0.9189
Epoch 245/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.1255 - accuracy: 0.9459
Epoch 246/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.1233 - accuracy: 0.9459
Epoch 247/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.1257 - accuracy: 0.9459
Epoch 248/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.1270 - accuracy: 0.9459
Epoch 249/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.1237 - accuracy: 0.9459
Epoch 250/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.1229 - accuracy: 0.9459
Epoch 251/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.1233 - accuracy: 0.

In [294]:
new = "Im have installed anaconda on my machine, but i don't know conda is not working"
new_sentence = preprocess_text(new)
b_of_w = np.array([bag_of_words(new_sentence,all_patterns)])

In [296]:
predicted_probs = Model.predict(b_of_w)
print(predicted_probs)

1/1 [==============================] - 0s 45ms/step
[[5.7965255e-05 1.1680947e-02 9.8826104e-01]]


In [297]:
predicted_label_index = np.argmax(predicted_probs)
predicted_tag = [tag for tag, index in label_to_index.items() if index == predicted_label_index][0]

print("Predicted tag:", predicted_tag)

Predicted tag: Python
